# "Data Engineering - Week 1"
> "This is week 1 from the Data Engineering Zoomcamp."

- toc: True
- branch: master
- badges: true
- comments: true
- categories: [data engineering, jupyter]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true

> youtube: https://youtu.be/bkJZDmreIpA

# Docker




> youtube: https://youtu.be/EYNwNlOrpr0

In [ ]:
# pipeline.py

import sys

import pandas as pd

print(sys.argv)

day = sys.argv[1]

# some fancy stuff with pandas

print(f'job finished successfully for day = {day}')

In [ ]:
# Dockerfile

FROM python:3.9.1

RUN pip install pandas

WORKDIR /app
COPY pipeline.py pipeline.py 

ENTRYPOINT [ "python", "pipeline.py" ]

In [ ]:
docker build -t test:pandas .

# PostgreSQL

> youtube: https://youtu.be/EYNwNlOrpr0

In [ ]:
docker run -it \
> -e POSTGRES_USER="root" \
> -e POSTGRES_PASSWORD="root" \
> -e POSTGRES_DB="ny_taxi" \
> -v $(pwd)/ny_taxi_postgres_data:/var/lib/postgresql/data \
> -p 5432:5432 \
> postgres:13

pgcli is a python package for connecting to the postgres database.

In [ ]:
! pip install pgcli

In [ ]:
pgcli -h localhost -p 5432 -u root -d ny_taxi